# Working with UP42 webhooks using the SDK

In the previous [article](https://up42.com/blog/tech/first-step-into-webhooks-no-code-required) we used the UP42 [console](https://console.up42.com) to manage webhooks. Now we use the [UP42 SDK](https://sdk.up42.com) to do.

**N.B.**: Make sure you are using a version  **$\ 0.22.0**. This is the version where support for webhooks was introduced.

## Installing the UP42 Python SDK

The module is called `up42`. Python **>= 3.8** is **required**.

 1. Create a virtual environment.
```bash
 mkvirtualenv --python=$(which python3) up42-py
```
 2. Activate the environment.
```bash
 workon up42-py
```
 3. Install the module.
 ```bash
 pip install up42-py
 ```
 4. Install Jupyter Lab.
```bash
 pip install jupyterlab
 ```
 5. Done.
 
Now we can just import it.

In [ ]:
import up42

## Authenticate with project ID and project key
The project API key and the project ID are read from a JSON file. This file has the following structure:
```
{
  "project_id": "<your-project-ID-here>",
  "project_api_key": "<your-api-key-here>"
}
```
To find out how to obtain the project API key and project ID the please refer to the [documentation].(https://docs.up42.com/processing-platform/projects#developers)

Reading the credentials from the JSON file `~/.up42/proj_default.json`.

In [ ]:
from pathlib import Path

project_config_file = Path.home() /  ".up42" / "proj_default.json"
up42.authenticate(cfg_file = project_config_file)

## Getting the webhooks under your account

We just invoke the [`get_webhooks`](https://sdk.up42.com/reference/webhooks-reference/) function (method).

In [ ]:
webhooks = up42.get_webhooks()

## Getting the (name, id) tuples

We want now to manipulate a specific webhook. For that we build a list of `(<name>, <id>)` tuples for each webhook.

In [ ]:
wh_pairs = list(map(lambda e: (e["name"], e["id"]), [w.info for w in webhooks]))

In [ ]:
wh_pairs

We're **interested** in the webhook called *Jobs webhook example*.

In [ ]:
wh_pairs[-1]

Now taking a closer look.

In [ ]:
current_wh = webhooks[-1]

In [ ]:
current_wh.__dict__

Let us now see what methods we have available. We are going to use a small module with helper functions.

We want to list all the non-magic methods that this class posesses. Somewhat similar to Julia's [methodswith](https://docs.julialang.org/en/v1/stdlib/InteractiveUtils/#InteractiveUtils.methodswith).

In [ ]:
import pynb_helpers.misc as misc

misc.methods(type(current_wh))

## Testing the webhook

Before we proceed we want to [test](https://docs.up42.com/developers/api#operation/runWebhookTests) the webhook. We already know the name and signature of the function. Is named `trigger_test_events` and takes no arguments.

In [ ]:
current_wh.trigger_test_events()

In [ ]:
import pynb_helpers.misc as misc

In [ ]:
misc.methods(type(current_wh))

## Updating the webhook to also notify us of orders

We see that the current webhook only works for jobs.



In [ ]:
current_wh.__dict__["_info"]["events"]

We want to make it work also for orders so we need to update the webhook by adding the order status event. First let's see the available events.

In [ ]:
up42.get_webhook_events()

So we need to add the `"order.status"` event to current webhook list of events while keeping all other parameters the same, except for the name that we also want to change to something more meaningful in our current context.

In [ ]:
current_wh.update(name="My webhook setup with ngrok", 
                  url=current_wh.__dict__["_info"]["url"],
                  events=["job.status", "order.status"],
                  active=True, 
                  secret= current_wh.__dict__["_info"]["secret"])

Let us now see the list of events again.

In [ ]:
current_wh.__dict__["_info"]["events"]

Now we can test the webhook again and we should get also _simulated_ order status events.

In [ ]:
current_wh.trigger_test_events()

We can see that we have now also order statuses related events.

## Going further

Of course we could use any of the other additional methods: `update`, `delete`. Or even [create](https://sdk.up42.com/webhooks/#create-a-new-webhook) a new webhook. Of course the later is a method of the `up42` class.